<a href="https://colab.research.google.com/github/majeed76/Renewable-System-Optimization/blob/main/Average_Model/notebooks/Average_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# 1) قرّاء بيانات الشمس والطلب
solar_url  = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/Timeseries_2022%20(1).csv"
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"

df_solar = (
    pd.read_csv(solar_url, skiprows=10, sep=",", engine='python', on_bad_lines='skip')
      .assign(time=lambda d: pd.to_datetime(d['time'], format='%Y%m%d:%H%M', errors='coerce'))
      .dropna(subset=['time'])
)
# نجعل G(i) و WS10m أرقام
df_solar['G(i)']    = pd.to_numeric(df_solar['G(i)'], errors='coerce').fillna(0)
df_solar['WS10m']  = pd.to_numeric(df_solar['WS10m'], errors='coerce').fillna(0)

# 2) حساب المتوسط اليومي والشّاحني للirradiance
df_solar['date']   = df_solar['time'].dt.date
avg_daily_irr = df_solar.groupby('date')['G(i)'].mean().mean()       # W/m²·h/day
avg_hourly_irr = df_solar['G(i)'].mean()                             # W/m²·h

# 3) قرّاء بيانات الطلب
df_demand = pd.read_csv(demand_url, sep=";")
df_demand["MWh"] = pd.to_numeric(df_demand["Consommation annuelle totale de l'adresse (MWh)"],
                                  errors='coerce').fillna(0)
total_kWh      = df_demand["MWh"].sum() * 1000
avg_daily_kWh  = total_kWh / 365
avg_hourly_kWh = total_kWh / 8760

# 4) معطيات التوربينات
rho       = 1.225      # air density kg/m³
rotor_d   = 117        # rotor diameter m (Vestas V117)
A_rotor   = 3.1416*(rotor_d/2)**2  # m²
Cp        = 0.4        # capacity factor (مثال)

# 5) متوسط إنتاج الرياح بالساعة (kWh)
# E_wind (kW) = 0.5 * ρ * A * v³ * Cp  -> نضرب بساعة لتحويل kW·h
avg_wind_power_kW = 0.5 * rho * A_rotor * (df_solar['WS10m']**3).mean() * Cp / 1000
avg_wind_kWh = avg_wind_power_kW * 1    # average hourly

# 6) نعرض النتائج
print(f"☀️ Avg hourly solar irradiance: {avg_hourly_irr:.2f} W/m²·h")
print(f"📅 Avg daily solar irradiance: {avg_daily_irr:.2f} W/m²·h/day")
print(f"🌬️ Avg wind power: {avg_wind_kWh:.2f} kWh/hour")
print(f"🔌 Total annual demand: {total_kWh:,.0f} kWh/year")
print(f"📅 Avg daily demand: {avg_daily_kWh:,.2f} kWh/day")
print(f"⏰ Avg hourly demand: {avg_hourly_kWh:,.2f} kWh/hour")


☀️ Avg hourly solar irradiance: 169.31 W/m²·h
📅 Avg daily solar irradiance: 169.31 W/m²·h/day
🌬️ Avg wind power: 142.28 kWh/hour
🔌 Total annual demand: 1,827,071,546 kWh/year
📅 Avg daily demand: 5,005,675.47 kWh/day
⏰ Avg hourly demand: 208,569.81 kWh/hour


In [ ]:
# === 2. Load electricity demand data ===
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"
df_demand = pd.read_csv(demand_url, sep=";")

# Convert annual consumption to numeric (in MWh)
df_demand["Consommation annuelle totale de l'adresse (MWh)"] = pd.to_numeric(
    df_demand["Consommation annuelle totale de l'adresse (MWh)"], errors='coerce'
)
df_demand = df_demand.dropna(subset=["Consommation annuelle totale de l'adresse (MWh)"])

# Total and average demand
total_annual_kWh = df_demand["Consommation annuelle totale de l'adresse (MWh)"].sum() * 1000
average_daily_kWh = total_annual_kWh / 365
average_hourly_kWh = total_annual_kWh / 8760

In [ ]:
# === 3. Display results ===
print("☀️ Average hourly solar irradiance:", round(average_hourly_irradiance, 2), "W/m²/hour")
print("📅 Average daily solar irradiance:", round(average_daily_irradiance, 2), "W/m²/day")
print("🔌 Total annual electricity demand:", round(total_annual_kWh, 2), "kWh/year")
print("📅 Average daily electricity demand:", round(average_daily_kWh, 2), "kWh/day")
print("⏰ Average hourly electricity demand:", round(average_hourly_kWh, 2), "kWh/hour")

☀️ Average hourly solar irradiance: 169.31 W/m²/hour
📅 Average daily solar irradiance: 169.31 W/m²/day
🔌 Total annual electricity demand: 1827071546.0 kWh/year
📅 Average daily electricity demand: 5005675.47 kWh/day
⏰ Average hourly electricity demand: 208569.81 kWh/hour


In [2]:
!pip install scikit-optimize
from google.colab import output
output.enable_custom_widget_manager()
from google.colab import output
output.disable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.7 MB/s eta 0:00:00


In [9]:
# 1) استيراد المكتبات
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
from IPython.display import display

# 2) تحميل ومعالجة البيانات (كما عندك)
solar_url  = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/Timeseries_2022%20(1).csv"
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"

df_solar = (
    pd.read_csv(solar_url, skiprows=10, sep=",", engine='python', on_bad_lines='skip')
      .assign(time=lambda d: pd.to_datetime(d['time'], format='%Y%m%d:%H%M', errors='coerce'))
      .dropna(subset=['time'])
)
df_solar['G(i)'] = pd.to_numeric(df_solar['G(i)'], errors='coerce').fillna(0)

df_demand = pd.read_csv(demand_url, sep=";")
df_demand["MWh"] = pd.to_numeric(
    df_demand["Consommation annuelle totale de l'adresse (MWh)"],
    errors='coerce'
).fillna(0)
total_kWh      = df_demand["MWh"].sum() * 1000
avg_hourly_kWh = total_kWh / 8760
df_solar['demand_kWh'] = avg_hourly_kWh

panel_area, panel_eff, batt_eff = 2.56, 0.156, 0.9

def unmet_hours(panels, capacity):
    prod    = panels * panel_area * panel_eff * df_solar['G(i)'] / 1000
    surplus = prod - df_solar['demand_kWh']
    batt, unmet = 0.0, 0
    for s in surplus:
        if s >= 0:
            batt += s * batt_eff
        else:
            if batt + s >= 0:
                batt += s
            else:
                batt, unmet = 0.0, unmet + 1
        batt = min(batt, capacity)
    return unmet

# 3) إنشاء الرسم التفاعلي مرة واحدة
fig = go.FigureWidget(
    data=[go.Scatter(
        x=[], y=[],
        mode='markers',
        marker=dict(size=[], color=[], colorscale='Viridis', showscale=True),
    )],
    layout=go.Layout(
        title="Bayesian Optimization Progress",
        xaxis_title='PV Panels',
        yaxis_title='Storage (kWh)',
    )
)
display(fig)

# 4) دالة الهدف
space = [
    Integer(1_000, 5_000_000,   name='panels'),
    Integer(0,     10_000_000, name='capacity_kWh'),
]

@use_named_args(space)
def objective(**p):
    u = unmet_hours(p['panels'], p['capacity_kWh'])
    penalty = 0.05 * p['capacity_kWh'] / 1000
    return u + penalty

# 5) callback لإضافة نقطة لكل تكرار
def plot_callback(res):
    p, c = res.x_iters[-1]
    val  = res.func_vals[-1]
    scatter = fig.data[0]
    scatter.x += (p,)
    scatter.y += (c,)
    scatter.marker.size += (8,)
    scatter.marker.color += (val,)

# 6) تشغيل GP minimizer مع callback
res = gp_minimize(
    func            = objective,
    dimensions      = space,
    acq_func        = "EI",
    n_calls         = 30,
    n_random_starts = 5,
    callback        = [plot_callback],
    random_state    = 0
)

best_p, best_c = res.x
best_u         = unmet_hours(best_p, best_c)
print(f"🔍 Best: panels={best_p}, storage={best_c} kWh, unmet_hours={best_u}")


FigureWidget({
    'data': [{'marker': {'color': [],
                         'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                        '#482878'], [0.2222222222222222,
                                        '#3e4989'], [0.3333333333333333,
                                        '#31688e'], [0.4444444444444444,
                                        '#26828e'], [0.5555555555555556,
                                        '#1f9e89'], [0.6666666666666666,
                                        '#35b779'], [0.7777777777777778,
                                        '#6ece58'], [0.8888888888888888,
                                        '#b5de2b'], [1.0, '#fde725']],
                         'showscale': True,
                         'size': []},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '994e5119-aff3-41d7-96fd-b7a2887d1ab9',
              'x': [],
              'y': []}],
    'layout': {'template': '.

🔍 Best: panels=5000000, storage=6514847 kWh, unmet_hours=1237


In [12]:
# 2) imports
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
from IPython.display import display

# 3) load & preprocess (same as before)
solar_url  = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/Timeseries_2022%20(1).csv"
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"

df_solar = (
    pd.read_csv(solar_url, skiprows=10, sep=",", engine='python', on_bad_lines='skip')
      .assign(time=lambda d: pd.to_datetime(d['time'], format='%Y%m%d:%H%M', errors='coerce'))
      .dropna(subset=['time'])
)
df_solar['G(i)']  = pd.to_numeric(df_solar['G(i)'], errors='coerce').fillna(0)
df_solar['WS10m'] = pd.to_numeric(df_solar['WS10m'], errors='coerce').fillna(0)

df_demand = pd.read_csv(demand_url, sep=";")
df_demand["MWh"] = pd.to_numeric(
    df_demand["Consommation annuelle totale de l'adresse (MWh)"],
    errors='coerce'
).fillna(0)
total_kWh      = df_demand["MWh"].sum() * 1000
avg_hourly_kWh = total_kWh / 8760
df_solar['demand_kWh'] = avg_hourly_kWh

# 4) fixed params
panel_area, panel_eff, batt_eff = 2.56, 0.156, 0.9
rho = 1.225
rotor_d = 117
A_rotor = np.pi*(rotor_d/2)**2
Cp = 0.4

def unmet_hours(params):
    panels, turbines, capacity = params
    prod_s = panels * panel_area * panel_eff * df_solar['G(i)'] / 1000
    prod_w = turbines * 0.5 * rho * A_rotor * (df_solar['WS10m']**3) * Cp / 1000
    surplus = (prod_s + prod_w) - df_solar['demand_kWh']
    batt = unmet = 0
    for s in surplus:
        if s >= 0:
            batt += s * batt_eff
        else:
            if batt + s >= 0:
                batt += s
            else:
                unmet += 1
                batt = 0
        batt = min(batt, capacity)
    return unmet

# 5) search space
space = [
    Integer( 1_000, 50_000,    name='panels'),
    Integer(     0,  1_000,    name='turbines'),
    Integer(     0, 10_000_000, name='capacity'),
]

# 6) build a larger, cleaner 3D FigureWidget
fig = go.FigureWidget(
    data=[go.Scatter3d(
        x=[], y=[], z=[],
        mode='markers',
        marker=dict(
            size=6,
            color=[],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Obj<br>(unmet+penalty)', lenmode='fraction', len=0.6),
            line=dict(width=1, color='DarkSlateGrey'),
            opacity=0.8
        )
    )],
    layout=go.Layout(
        title=dict(text="🔍 BO Progress: Panels ↔ Turbines ↔ Storage", x=0.5, font=dict(size=20)),
        scene=dict(
            xaxis=dict(title='PV Panels', titlefont=dict(size=14), tickfont=dict(size=12)),
            yaxis=dict(title='Wind Turbines', titlefont=dict(size=14), tickfont=dict(size=12)),
            zaxis=dict(title='Storage (kWh)', titlefont=dict(size=14), tickfont=dict(size=12)),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        width=900,
        height=700,
    )
)
display(fig)

# 7) callback to append each new point in place
def plot_cb(res):
    p,t,c = res.x_iters[-1]
    val   = res.func_vals[-1]
    sc = fig.data[0]
    sc.x += (p,)
    sc.y += (t,)
    sc.z += (c,)
    sc.marker.color += (val,)

# 8) objective with small storage penalty
@use_named_args(space)
def objective(**kw):
    return unmet_hours((kw['panels'], kw['turbines'], kw['capacity'])) + 0.05*kw['capacity']/1000

# 9) run GP
res = gp_minimize(
    func            = objective,
    dimensions      = space,
    acq_func        = "EI",
    n_calls         = 30,
    n_random_starts = 5,
    callback        = [plot_cb],
    random_state    = 0
)

best = res.x
print(f"🎯 Best → Panels={best[0]}, Turbines={best[1]}, Storage={best[2]} kWh, Unmet={unmet_hours(best)}")


FigureWidget({
    'data': [{'marker': {'color': [],
                         'colorbar': {'len': 0.6, 'lenmode': 'fraction', 'title': {'text': 'Obj<br>(unmet+penalty)'}},
                         'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                        '#482878'], [0.2222222222222222,
                                        '#3e4989'], [0.3333333333333333,
                                        '#31688e'], [0.4444444444444444,
                                        '#26828e'], [0.5555555555555556,
                                        '#1f9e89'], [0.6666666666666666,
                                        '#35b779'], [0.7777777777777778,
                                        '#6ece58'], [0.8888888888888888,
                                        '#b5de2b'], [1.0, '#fde725']],
                         'line': {'color': 'DarkSlateGrey', 'width': 1},
                         'opacity': 0.8,
                         'showscale': True,
         

🎯 Best → Panels=50000, Turbines=1000, Storage=8772632 kWh, Unmet=5478


Support for third party widgets will remain active for the duration of the session. To disable support: